In [1]:
import numpy as np
from scipy.linalg import eigh, qr, null_space
import matplotlib.pyplot as plt
from scipy.sparse import kron, identity, csr_matrix, lil_matrix, dok_matrix, coo_matrix, issparse
from scipy.sparse.linalg import eigsh, eigs
from scipy.special import factorial, comb
from scipy.optimize import curve_fit
from qutip import Qobj, ptrace, entropy_vn, qeye, tensor
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed
from itertools import combinations
#from quspin.basis import spin_basis_1d, spin_basis_general
import tenpy as tp

In [2]:
# spinless  jw

# |0> and |1> states for spinless fermions
ket_zero = csr_matrix([[1], [0]], dtype=np.complex128)
ket_one = csr_matrix([[0], [1]], dtype=np.complex128)

# Single-site operators
# Creation operator c†
c_dag = csr_matrix([[0, 0], [1, 0]], dtype=np.complex128)

# Annihilation operator c
c = csr_matrix([[0, 1], [0, 0]], dtype=np.complex128)

# Number operator n = c†c
n = csr_matrix([[0, 0], [0, 1]], dtype=np.complex128)

# Identity operator
I_2 = identity(2, format='csr', dtype=np.complex128)

# Pauli Z operator (for Jordan-Wigner string)
sigma_z = csr_matrix([[1, 0], [0, -1]], dtype=np.complex128)

# --- Multi-site operators ---
def vacuum_state(L):
    """Create vacuum state |0>^⊗L for L sites"""
    state = ket_zero
    for _ in range(L-1):
        state = kron(state, ket_zero, format='csr')
    return state

def full_operator_spinless(L, op, site):
    """
    Create full fermionic operator with Jordan-Wigner string
    Args:
        L: number of sites
        op: single-site operator (c, c_dag, or n)
        site: site index (0 to L-1)
    """
    if site >= L or site < 0:
        raise ValueError(f"Site index {site} out of range for {L} sites")
    
    result = 1
    for i in range(L):
        if i < site:
            # Jordan-Wigner string: σ_z for all sites to the left
            result = kron(result, sigma_z, format='csr')
        elif i == site:
            # Apply the operator at the target site
            result = kron(result, op, format='csr')
        else:
            # Identity for sites to the right
            result = kron(result, I_2, format='csr')
    return result

def creation_operator(L, site):
    """Creation operator c†_i with Jordan-Wigner string"""
    return full_operator_spinless(L, c_dag, site)

def annihilation_operator(L, site):
    """Annihilation operator c_i with Jordan-Wigner string"""
    return full_operator_spinless(L, c, site)

def number_operator(L, site):
    """Number operator n_i = c†_i c_i"""
    return full_operator_spinless(L, n, site)

def total_number_operator(L):
    """Total number operator N = Σ_i n_i"""
    N_total = csr_matrix((2**L, 2**L), dtype=np.complex128)
    for i in range(L):
        N_total += number_operator(L, i)
    return N_total

# --- Check anticommutation relations ---
def check_anticommutation_spinless(L, site_i, site_j):
    """Check anticommutators for spinless fermion operators"""
    ci = annihilation_operator(L, site_i)
    ci_dag = creation_operator(L, site_i)
    cj = annihilation_operator(L, site_j)
    cj_dag = creation_operator(L, site_j)
    
    results = {}
    
    # Same site: {c_i, c†_i} = 1
    anticomm_same = ci @ ci_dag + ci_dag @ ci
    results[f"{{c_{site_i}, c†_{site_i}}}"] = anticomm_same.todense()
    
    # Different sites: {c_i, c†_j} = 0 (i ≠ j)
    if site_i != site_j:
        anticomm_diff = ci @ cj_dag + cj_dag @ ci
        results[f"{{c_{site_i}, c†_{site_j}}}"] = anticomm_diff.todense()
        
        # {c_i, c_j} = 0
        anticomm_cc = ci @ cj + cj @ ci
        results[f"{{c_{site_i}, c_{site_j}}}"] = anticomm_cc.todense()
        
        # {c†_i, c†_j} = 0
        anticomm_cdagcdag = ci_dag @ cj_dag + cj_dag @ ci_dag
        results[f"{{c†_{site_i}, c†_{site_j}}}"] = anticomm_cdagcdag.todense()
    
    return results


In [3]:
# check anticommutation relations for L sites

L = 4

anticom_results = check_anticommutation_spinless(L, 0, 1)

for key, value in anticom_results.items():
    print(f"{key}: {np.real(np.round(value, 5))}")

{c_0, c†_0}: [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
{c_0, c†_1}: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0.

In [ ]:
# spinful jw - corrected and verified

# Four basis states for each site: |0>, |↑>, |↓>, |↑↓>
ket_empty = csr_matrix([[1], [0], [0], [0]], dtype=np.complex128)  # |0>
ket_up = csr_matrix([[0], [1], [0], [0]], dtype=np.complex128)     # |↑>
ket_down = csr_matrix([[0], [0], [1], [0]], dtype=np.complex128)   # |↓>
ket_both = csr_matrix([[0], [0], [0], [1]], dtype=np.complex128)   # |↑↓>

# Single-site operators for spin-up
# Annihilation operator c_↑
c_up = csr_matrix([
    [0, 1, 0, 0],  # |0> -> 0
    [0, 0, 0, 0],  # |↑> -> |0>
    [0, 0, 0, 1],  # |↓> -> 0
    [0, 0, 0, 0]   # |↑↓> -> |↓>
], dtype=np.complex128)

# Creation operator c†_↑
c_up_dag = c_up.getH()

# Single-site operators for spin-down
# Annihilation operator c_↓
c_down = csr_matrix([
    [0, 0, 1, 0],   # |0> -> 0
    [0, 0, 0, -1],   # |↑> -> 0
    [0, 0, 0, 0],   # |↓> -> |0>
    [0, 0, 0, 0]   # |↑↓> -> -|↑> (anticommutation with c†_↑)
], dtype=np.complex128)

# Creation operator c†_↓
c_down_dag = c_down.getH()

# Number operators
n_up = c_up_dag @ c_up
n_down = c_down_dag @ c_down
n_total_site = n_up + n_down

# Identity operator (4x4 for spinful sites)
I_4 = identity(4, format='csr', dtype=np.complex128)

# Parity operator P = (-1)^n for Jordan-Wigner string
P = csr_matrix([
    [1, 0, 0, 0],   # |0> -> +|0>
    [0, -1, 0, 0],  # |↑> -> -|↑>
    [0, 0, -1, 0],  # |↓> -> -|↓>
    [0, 0, 0, 1]    # |↑↓> -> +|↑↓>
], dtype=np.complex128)

# --- Verification function ---
def verify_spinful_operators():
    """Verify the action of operators on the 4 basis states"""
    print("=== Verification of Spinful Operators ===")
    
    states = [ket_empty, ket_up, ket_down, ket_both]
    state_names = ["|0>", "|↑>", "|↓>", "|↑↓>"]
    
    print("\n1. Spin-up creation operator c†_↑:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = c_up_dag @ state
        print(f"   c†_↑ {name} = {result.toarray().flatten()}")
    
    print("\n2. Spin-up annihilation operator c_↑:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = c_up @ state
        print(f"   c_↑ {name} = {result.toarray().flatten()}")
    
    print("\n3. Spin-down creation operator c†_↓:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = c_down_dag @ state
        print(f"   c†_↓ {name} = {result.toarray().flatten()}")
    
    print("\n4. Spin-down annihilation operator c_↓:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = c_down @ state
        print(f"   c_↓ {name} = {result.toarray().flatten()}")
    
    print("\n5. Number operators:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        n_up_result = (n_up @ state).toarray().flatten()
        n_down_result = (n_down @ state).toarray().flatten()
        n_total_result = (n_total_site @ state).toarray().flatten()
        print(f"   n_↑ {name} = {n_up_result}")
        print(f"   n_↓ {name} = {n_down_result}")
        print(f"   n_total {name} = {n_total_result}")
    
    print("\n6. Parity operator P:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = (P @ state).toarray().flatten()
        print(f"   P {name} = {result}")
    
    print("\n7. Verify anticommutation on same site:")
    # {c_↑, c†_↓} = 0 and {c_↓, c†_↑} = 0
    anticomm1 = c_up @ c_down_dag + c_down_dag @ c_up
    anticomm2 = c_down @ c_up_dag + c_up_dag @ c_down
    print(f"   {{c_↑, c†_↓}} = \n{anticomm1.toarray()}")
    print(f"   {{c_↓, c†_↑}} = \n{anticomm2.toarray()}")
    
    # {c_↑, c†_↑} = 1 and {c_↓, c†_↓} = 1
    anticomm3 = c_up @ c_up_dag + c_up_dag @ c_up
    anticomm4 = c_down @ c_down_dag + c_down_dag @ c_down
    print(f"   {{c_↑, c†_↑}} = \n{anticomm3.toarray()}")
    print(f"   {{c_↓, c†_↓}} = \n{anticomm4.toarray()}")

# Run verification
verify_spinful_operators()

=== Verification of Spinful Operators ===

1. Spin-up creation operator c†_↑:
   c†_↑ |0> = [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
   c†_↑ |↑> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c†_↑ |↓> = [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
   c†_↑ |↑↓> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]

2. Spin-up annihilation operator c_↑:
   c_↑ |0> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↑ |↑> = [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↑ |↓> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↑ |↑↓> = [0.+0.j 0.+0.j 1.+0.j 0.+0.j]

3. Spin-down creation operator c†_↓:
   c†_↓ |0> = [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
   c†_↓ |↑> = [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
   c†_↓ |↓> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c†_↓ |↑↓> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]

4. Spin-down annihilation operator c_↓:
   c_↓ |0> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↓ |↑> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↓ |↓> = [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↓ |↑↓> = [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]

5. Number operators:
   n_↑ |0> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   n_↓ |0> = [0.+0.j 0.+0.j 0.+

In [ ]:
# spinful jw - generalized from spinless case

# --- Multi-site operators ---
def vacuum_state_spinful(L):
    """Create vacuum state |0>^⊗L for L sites"""
    state = ket_empty
    for _ in range(L-1):
        state = kron(state, ket_empty, format='csr')
    return state

def full_operator_spinful(L, op, site):
    """
    Create full fermionic operator with Jordan-Wigner string
    Args:
        L: number of sites
        op: single-site operator (c_up, c_up_dag, c_down, c_down_dag, etc.)
        site: site index (0 to L-1)
    """
    if site >= L or site < 0:
        raise ValueError(f"Site index {site} out of range for {L} sites")
    
    result = 1
    for i in range(L):
        if i < site:
            # Jordan-Wigner string: parity operator for all sites to the left
            result = kron(result, P, format='csr')
        elif i == site:
            # Apply the operator at the target site
            result = kron(result, op, format='csr')
        else:
            # Identity for sites to the right
            result = kron(result, I_4, format='csr')
    return result

def creation_operator_up(L, site):
    """Creation operator c†_↑,i with Jordan-Wigner string"""
    return full_operator_spinful(L, c_up_dag, site)

def annihilation_operator_up(L, site):
    """Annihilation operator c_↑,i with Jordan-Wigner string"""
    return full_operator_spinful(L, c_up, site)

def creation_operator_down(L, site):
    """Creation operator c†_↓,i with Jordan-Wigner string"""
    return full_operator_spinful(L, c_down_dag, site)

def annihilation_operator_down(L, site):
    """Annihilation operator c_↓,i with Jordan-Wigner string"""
    return full_operator_spinful(L, c_down, site)

def number_operator_up(L, site):
    """Number operator n_↑,i = c†_↑,i c_↑,i"""
    return full_operator_spinful(L, n_up, site)

def number_operator_down(L, site):
    """Number operator n_↓,i = c†_↓,i c_↓,i"""
    return full_operator_spinful(L, n_down, site)

def number_operator_total_site(L, site):
    """Total number operator n_i = n_↑,i + n_↓,i"""
    return full_operator_spinful(L, n_total_site, site)

def total_number_operator_spinful(L):
    """Total number operator N = Σ_i (n_↑,i + n_↓,i)"""
    N_total = csr_matrix((4**L, 4**L), dtype=np.complex128)
    for i in range(L):
        N_total += number_operator_up(L, i)
        N_total += number_operator_down(L, i)
    return N_total

# --- Check anticommutation relations ---
def check_anticommutation_spinful(L, site_i, site_j):
    """Check anticommutators for spinful fermion operators"""
    # Get all operators
    ci_up = annihilation_operator_up(L, site_i)
    ci_up_dag = creation_operator_up(L, site_i)
    ci_down = annihilation_operator_down(L, site_i)
    ci_down_dag = creation_operator_down(L, site_i)
    
    cj_up = annihilation_operator_up(L, site_j)
    cj_up_dag = creation_operator_up(L, site_j)
    cj_down = annihilation_operator_down(L, site_j)
    cj_down_dag = creation_operator_down(L, site_j)
    
    results = {}
    
    # Same site, same spin: {c_σ,i, c†_σ,i} = 1
    anticomm_same_up = ci_up @ ci_up_dag + ci_up_dag @ ci_up
    anticomm_same_down = ci_down @ ci_down_dag + ci_down_dag @ ci_down
    results[f"{{c_↑,{site_i}, c†_↑,{site_i}}}"] = anticomm_same_up.todense()
    results[f"{{c_↓,{site_i}, c†_↓,{site_i}}}"] = anticomm_same_down.todense()
    
    if site_i != site_j:
        # Different sites, same spin: {c_σ,i, c†_σ,j} = 0
        anticomm_diff_up = ci_up @ cj_up_dag + cj_up_dag @ ci_up
        anticomm_diff_down = ci_down @ cj_down_dag + cj_down_dag @ ci_down
        results[f"{{c_↑,{site_i}, c†_↑,{site_j}}}"] = anticomm_diff_up.todense()
        results[f"{{c_↓,{site_i}, c†_↓,{site_j}}}"] = anticomm_diff_down.todense()
        
        # Different sites, different spins: {c_σ,i, c†_σ',j} = 0
        anticomm_diff_mixed1 = ci_up @ cj_down_dag + cj_down_dag @ ci_up
        anticomm_diff_mixed2 = ci_down @ cj_up_dag + cj_up_dag @ ci_down
        results[f"{{c_↑,{site_i}, c†_↓,{site_j}}}"] = anticomm_diff_mixed1.todense()
        results[f"{{c_↓,{site_i}, c†_↑,{site_j}}}"] = anticomm_diff_mixed2.todense()
    
    # Same site, different spins: {c_σ,i, c†_σ',i} = 0
    anticomm_same_mixed1 = ci_up @ ci_down_dag + ci_down_dag @ ci_up
    anticomm_same_mixed2 = ci_down @ ci_up_dag + ci_up_dag @ ci_down
    results[f"{{c_↑,{site_i}, c†_↓,{site_i}}}"] = anticomm_same_mixed1.todense()
    results[f"{{c_↓,{site_i}, c†_↑,{site_i}}}"] = anticomm_same_mixed2.todense()
    
    return results

def create_eta_states(L,n):
    """
    Create Fock state by applying creation operators to vacuum
    Args:
        L: number of sites
        occupied_sites_up: list of site indices to occupy with spin-up
        occupied_sites_down: list of site indices to occupy with spin-down
    """
    state = vacuum_state_spinful(L)
    
    # Apply spin-up creation operators first
    for site in sorted(n):
        c_up_dag_i = creation_operator_up(L, site)
        state = c_up_dag_i @ state
    
    # Apply spin-down creation operators
    for site in sorted(n):
        c_down_dag_i = creation_operator_down(L, site)
        state = c_down_dag_i @ state
    
    return state

# --- Hubbard Hamiltonian ---
def hubbard_hamiltonian(L, t=1.0, U=1.0, pbc=False):
    """
    Create Hubbard Hamiltonian H = -t Σ_{i,σ} (c†_{i,σ} c_{i+1,σ} + h.c.) + U Σ_i n_{i,↑} n_{i,↓}
    Args:
        L: number of sites
        t: hopping amplitude
        U: on-site interaction strength
        pbc: periodic boundary conditions
    """
    H = csr_matrix((4**L, 4**L), dtype=np.complex128)
    
    # Hopping terms
    max_i = L if pbc else L-1
    for i in range(max_i):
        j = (i + 1) % L
        
        # Spin-up hopping
        ci_up_dag = creation_operator_up(L, i)
        cj_up = annihilation_operator_up(L, j)
        H += -t * (ci_up_dag @ cj_up + cj_up.getH() @ ci_up_dag.getH())
        
        # Spin-down hopping
        ci_down_dag = creation_operator_down(L, i)
        cj_down = annihilation_operator_down(L, j)
        H += -t * (ci_down_dag @ cj_down + cj_down.getH() @ ci_down_dag.getH())
    
    # On-site interaction terms
    for i in range(L):
        ni_up = number_operator_up(L, i)
        ni_down = number_operator_down(L, i)
        H += U * (ni_up @ ni_down)
    
    return H

# --- Test function ---
def test_spinful_jw(L=2):
    """Test the spinful Jordan-Wigner implementation"""
    print(f"Testing spinful JW for L={L} sites")
    print("="*50)
    
    # Test anticommutation relations
    print("Anticommutation relations:")
    results = check_anticommutation_spinful(L, 0, 1 if L > 1 else 0)
    for key, val in list(results.items())[:6]:  # Show first 6 results
        max_val = np.max(np.abs(val))
        print(f"{key}: max |element| = {max_val:.2e}")
    
    # Create some Fock states
    print(f"\nFock states:")
    vacuum = vacuum_state_spinful(L)
    print(f"Vacuum |0...0>: shape {vacuum.shape}")
    
    single_up = create_fock_state_spinful(L, [0], [])
    print(f"Single spin-up at site 0: shape {single_up.shape}")
    
    if L >= 2:
        mixed_state = create_fock_state_spinful(L, [0], [1])
        print(f"Spin-up at site 0, spin-down at site 1: shape {mixed_state.shape}")
    
    # Test Hamiltonian
    H = hubbard_hamiltonian(L, t=1.0, U=2.0, pbc=False)
    print(f"\nHubbard Hamiltonian: shape {H.shape}, nnz = {H.nnz}")
    
    return True

In [6]:
# check anticommutation relations for L sites

L = 4

anticom_results = check_anticommutation_spinful(L, 0, 1)

for key, value in anticom_results.items():
    print(f"{key}: {np.real(np.round(value, 5))}")

{c_↑,0, c†_↑,0}: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
{c_↓,0, c†_↓,0}: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
{c_↑,0, c†_↑,1}: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{c_↓,0, c†_↓,1}: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{c_↑,0, c†_↓,1}: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{c_↓,0, c†_↑,1}: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ..